Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [11]:
import pandas as pd
wage = pd.read_csv("wage.csv")
wage

,person_id,gender,wage
0,0,1,46793.603811
1,1,1,33481.575720
2,2,1,44523.699084
3,3,1,15995.576829
4,4,0,10282.631224
...,...,...,...
1001,995,1,66503.737185
1002,996,1,9972.956272
1003,997,0,104504.616392
1004,998,1,98927.903076


3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [13]:
wage.loc[wage.gender == 0, ["gender"]] = "F"
wage.loc[wage.gender == 1, ["gender"]] = "M"

wage

,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [24]:
g = wage.groupby("gender", as_index=False)["wage"].agg({'mean'})
g

,gender,mean
0,F,40855.747261
1,M,46815.944005


5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз.
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [35]:
ind = wage.person_id.value_counts()
duplicates = ind[ind > 1].index

for pid in duplicates:
    unique_wages = wage[wage.person_id == pid].wage.nunique()
    print(f'person_id: {pid}, уникальных wage: {unique_wages}')

person_id: 17, уникальных wage: 1
person_id: 16, уникальных wage: 1
person_id: 15, уникальных wage: 1
person_id: 14, уникальных wage: 1
person_id: 13, уникальных wage: 1
person_id: 12, уникальных wage: 1


In [36]:
wage = wage.drop_duplicates(subset=['person_id', 'wage'])

In [37]:
ind_ = wage.person_id.value_counts()
ind_[ind_ > 1]

,count
person_id,


6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [38]:
wage['wage'].describe()

,wage
count,1000.000000
mean,43694.227404
std,55352.539343
min,-287418.645743
25%,14489.682367
50%,27309.529498
75%,52021.080258
max,755320.874132


In [39]:
wage = wage[wage['wage'] > 0]

7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [41]:
bonus = pd.read_csv("bonus.csv", sep=";")
bonus

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614
...,...,...
445,488,28102.252903
446,913,55549.347647
447,616,620397.407705
448,110,14086.067488


In [42]:
bonus.spl

AttributeError: 'DataFrame' object has no attribute 'spl'

8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [44]:
df = pd.merge(wage, bonus, how="outer")
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,995,M,66503.737185,3.452137e+03
995,996,M,9972.956272,3.892790e+05
996,997,F,104504.616392,5.380978e+04
997,998,M,98927.903076,NaN


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [57]:
df.bonus = df.bonus.fillna(0)
df = df[df['wage'].notnull()]
df["total"] = df.wage*12 + df.bonus
df
df.groupby("gender", as_index=False)["total"].agg({"Mean": 'mean', "Med": 'median'})

/tmp/ipython-input-1705551760.py:5: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df.groupby("gender", as_index=False)["total"].agg({"Mean": 'mean', "Med": 'median'})


,gender,Mean,Med
0,F,570746.139432,347622.913892
1,M,657142.490282,437499.824868


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [58]:
df.to_csv("df.csv", index=False)